In [41]:
import cv2
import dlib
import numpy as np

In [42]:
cap = cv2.VideoCapture(0)

In [43]:
face_detector = dlib.get_frontal_face_detector()

In [44]:
facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [45]:
_, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_detector(gray)

for face in faces:
    x1 = face.left()  # Left coordinate of the face rectangle
    y1 = face.top()  # Top coordinate of the face rectangle
    x2 = face.right()  # Width of the face rectangle
    y2 = face.bottom()
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    dark_threshold = 50 
    landmarks = facelandmark(gray, face)
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        cv2.circle(frame, (x, y), 1, (0, 255, 255), -1)
        # Convert landmarks to a list of tuples
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
        # Get the landmarks for the left and right eye
    left_eye_landmarks = landmarks[36:42]
    right_eye_landmarks = landmarks[42:48]
    
    # Find the bounding box for the left and right eye landmarks
    left_eye_bbox = cv2.boundingRect(np.array(left_eye_landmarks))
    right_eye_bbox = cv2.boundingRect(np.array(right_eye_landmarks))
    
    # Define the ROI for the area under the left eye
    left_eye_roi = gray[left_eye_bbox[1] + left_eye_bbox[3]//2 : left_eye_bbox[1] + left_eye_bbox[3], left_eye_bbox[0]: left_eye_bbox[0] + left_eye_bbox[2]]
    
    # Define the ROI for the area under the right eye
    right_eye_roi = gray[right_eye_bbox[1] + right_eye_bbox[3]//2 : right_eye_bbox[1] + right_eye_bbox[3], right_eye_bbox[0]: right_eye_bbox[0] + right_eye_bbox[2]]
    
    # Calculate the percentage of dark pixels under the left eye
    left_eye_dark_pixels = np.count_nonzero(left_eye_roi < dark_threshold)
    left_eye_total_pixels = left_eye_roi.shape[0] * left_eye_roi.shape[1]
    left_eye_percentage = (left_eye_dark_pixels / left_eye_total_pixels) * 100
    
    # Calculate the percentage of dark pixels under the right eye
    right_eye_dark_pixels = np.count_nonzero(right_eye_roi < dark_threshold)
    right_eye_total_pixels = right_eye_roi.shape[0] * right_eye_roi.shape[1]
    right_eye_percentage = (right_eye_dark_pixels / right_eye_total_pixels) * 100
    
    # Draw the percentage of dark pixels on the frame
    cv2.putText(frame, f"Left Eye Dark circle: {left_eye_percentage:.2f}%", (x1 - 200, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(frame, f"Right Eye Dark circle: {right_eye_percentage:.2f}%", (x1 - 200, y1 - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    
    # Detect face wrinkles
    face_wrinkle_score = abs(landmarks[27][1] - landmarks[8][1]) + abs(landmarks[19][1] - landmarks[24][1])

    # Normalize the face wrinkle score
    face_wrinkle_score /= abs(landmarks[8][1] - landmarks[24][1])
    
    # Get the vertical positions of the left eye landmarks
    left_eye_landmarks_y = [landmarks[i][1] for i in range(36, 42)]

    # Get the vertical positions of the right eye landmarks
    right_eye_landmarks_y = [landmarks[i][1] for i in range(42, 48)]

    # Calculate the average vertical position of the left eye landmarks
    left_eye_avg_y = sum(left_eye_landmarks_y) / len(left_eye_landmarks_y)

    # Calculate the average vertical position of the right eye landmarks
    right_eye_avg_y = sum(right_eye_landmarks_y) / len(right_eye_landmarks_y)

    # Calculate the eye wrinkle scores as the absolute difference between the average positions
    left_eye_wrinkle_score = abs(left_eye_avg_y - landmarks[37][1])
    right_eye_wrinkle_score = abs(right_eye_avg_y - landmarks[43][1])
    
    # Draw the face and eye wrinkles scores on the frame
    cv2.putText(frame, f"Face Wrinkle Score: {face_wrinkle_score:.2f}%", (x1 - 200, y1 - 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(frame, f"Left Eye Wrinkle Score: {left_eye_wrinkle_score:.2f}%", (x1 - 200, y1 - 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(frame, f"Right Eye Wrinkle Score: {right_eye_wrinkle_score:.2f}%", (x1 - 200, y1 - 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Set a range for dullness threshold
    dullness_min = 0
    dullness_max = 100
    # Calculate the average intensity of the face region
    face_roi = gray[y1:y2, x1:x2]
    face_avg_intensity = np.mean(face_roi)
    # Calculate the dullness score of the face region
    dullness_score = (face_avg_intensity - dullness_min) / (dullness_max - dullness_min) * 100

    # Set a threshold for dullness percentage
    dullness_threshold = 60

    # Check if the face is dull
    if dullness_score < dullness_threshold:
        cv2.putText(frame, "Dull Face", (x1 - 200, y1 - 130), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
        cv2.putText(frame, "Dullness is present", (x1 - 200, y1 - 160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    else:
        cv2.putText(frame, "No dullness", (x1 - 200, y1 - 160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    #dark sports
    dark_spot_threshold = 70 
    dark_spots = 0
     # Check if the point falls within the face region
    if x >= x1 and x <= x2 and y >= y1 and y <= y2:
        # Check if the pixel intensity is below the dark spot threshold
        if gray[y, x] < dark_spot_threshold:
            dark_spots += 1

    # Calculate the vertical distances between the upper and lower lip landmarks
    upper_lip_y = landmarks[62][1]
    lower_lip_y = landmarks[66][1]
    lip_distance = lower_lip_y - upper_lip_y

    # Calculate the vertical distances between the reference points for fully open mouth
    reference_upper_lip_y = landmarks[51][1]
    reference_lower_lip_y = landmarks[57][1]
    max_lip_distance = reference_lower_lip_y - reference_upper_lip_y

    # Normalize the lip distances using the maximum lip distance
    normalized_lip_distance = lip_distance / max_lip_distance

    # Calculate the hydration score as a percentage
    hydration_score = (1 - normalized_lip_distance) * 100

    # Draw the hydration score on the frame
    cv2.putText(frame, f"Hydration Score: {hydration_score:.2f}%", (x1 - 200, y1 - 200), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)


cv2.imshow("Frame", frame)
cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()


                  